In [1]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup

In [2]:
url="https://quotes.toscrape.com/page/1/"

In [3]:
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [5]:
try:
    response = requests.get(url, headers=header)
    response.raise_for_status()
    print("Request Successful")
    print(response.text[:2000])
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occured: {http_err}")

Request Successful
<!DOCTYPE html>
<html lang="en">
<head>
	<meta charset="UTF-8">
	<title>Quotes to Scrape</title>
    <link rel="stylesheet" href="/static/bootstrap.min.css">
    <link rel="stylesheet" href="/static/main.css">
    
    
</head>
<body>
    <div class="container">
        <div class="row header-box">
            <div class="col-md-8">
                <h1>
                    <a href="/" style="text-decoration: none">Quotes to Scrape</a>
                </h1>
            </div>
            <div class="col-md-4">
                <p>
                
                    <a href="/login">Login</a>
                
                </p>
            </div>
        </div>
    

<div class="row">
    <div class="col-md-8">

    <div class="quote" itemscope itemtype="http://schema.org/CreativeWork">
        <span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
        <span>by

In [8]:
soup = BeautifulSoup(response.text,"html.parser")

In [10]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Quotes to Scrape
  </title>
  <link href="/static/bootstrap.min.css" rel="stylesheet"/>
  <link href="/static/main.css" rel="stylesheet"/>
 </head>
 <body>
  <div class="container">
   <div class="row header-box">
    <div class="col-md-8">
     <h1>
      <a href="/" style="text-decoration: none">
       Quotes to Scrape
      </a>
     </h1>
    </div>
    <div class="col-md-4">
     <p>
      <a href="/login">
       Login
      </a>
     </p>
    </div>
   </div>
   <div class="row">
    <div class="col-md-8">
     <div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
      <span class="text" itemprop="text">
       “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
      </span>
      <span>
       by
       <small class="author" itemprop="author">
        Albert Einstein
       </small>
       <a href="/author/Albert

In [11]:
quotes = soup.find_all('div', class_="quote")

In [13]:
len(quotes)

10

In [14]:
quotes_list= []

In [15]:
for quote_div in quotes:
    quote_text_tag = quote_div.find("span", class_="text")
    author_tag = quote_div.find("small", class_="author")
    tags_tag = quote_div.find_all("a", class_="tag")

    if quote_text_tag and author_tag:
        quote_text = quote_text_tag.get_text(strip=True)
        author = author_tag.get_text(strip=True)
        tags = [tag.get_text(strip=True) for tag in tags_tag]

    quotes_list.append({
            "quote": quote_text,
            "author": author,
            "tags": tags
        })

In [16]:
df = pd.DataFrame(quotes_list)
df.head()

,quote,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"


In [18]:
# pip install textblob

In [25]:
from textblob import TextBlob
def analyze_textsentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    subjectivity = blob.sentiment.subjectivity
    sentiment = "😊Positive" if polarity > 0 else "☹️Negative" if polarity < 0 else "😶Neutral"
    return pd.Series([polarity, subjectivity, sentiment])

df[['polarity', 'subjectivity','sentiment']] = df['quote'].apply(analyze_textsentiment)

In [26]:
df.head()

,quote,author,tags,polarity,subjectivity,sentiment
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]",0.000000,0.000000,😶Neutral
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]",0.300000,0.750000,😊Positive
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]",0.003788,0.625000,😊Positive
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]",-0.050000,0.800000,☹️Negative
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]",-0.277778,0.833333,☹️Negative
